Задание 1. Загрузка данных
Изучить представленный набор данных на основе описания его столбцов в файле “horse_data.names” , загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Данные_фильмы_ДЗ1_Pandas/horse_data.csv", header=None, na_values='?')



In [ ]:
df.head(5)

In [ ]:
df.info()

по вышезапрошенной информации можно сделать вывод:
1. Данные имеют 300 строк и 28 столбцов;
2. 7 столбцов имеют числовой формат, 21 столбец имеет текстовый формат;
4. объем памяти - 65.5 КВ (необъемные данные).

In [ ]:
print(df.columns.tolist())

In [ ]:
# необходимо дать имена столбцам
df.columns = [ "surgery?", "age", "hospital_number", "rectal_temperature", "pulse", "respiratory_rate",
    "temperature_of_extremities", "peripheral_pulse", "mucous_membranes", "capillary_refill_time",
    "pain", "peristalsis", "abdominal_distension", "nasogastric_tube", "nasogastric_reflux",
    "nasogastric_reflux_ph", "rectal_examination", "abdomen", "packed_cell_volume", "total_protein",
    "abdominocentesis_appearance", "abdominocentesis_total_protein", "outcome",
    "surgical_lesion", "type_of_lesion_1", "type_of_lesion_2", "type_of_lesion_3", "cp_data"]

In [ ]:
print(df.columns.tolist()) # проверяем

Оставим столбцы, которые нам необходимы для дальнейшего анализа

In [ ]:
selected_columns=[
    "surgery?", "age", "rectal_temperature", "pulse", "respiratory_rate", "temperature_of_extremities",
    "pain", "outcome"
]
df1 = df[selected_columns].copy()




In [ ]:
df1

In [ ]:
# посмотрим базовые статистики для колич.данных
statistics = df1[['rectal_temperature', 'pulse', 'respiratory_rate']].describe()
print(statistics)

In [ ]:
# для категориальных данных
statistics2=df1[['surgery?','age', 'temperature_of_extremities', 'pain', 'outcome']].value_counts()
print(statistics2)

In [ ]:
statistics3=df1 [['surgery?','age', 'temperature_of_extremities', 'pain', 'outcome']].mode()
print(statistics3)

In [ ]:
for column in selected_columns:
    unique_values = df1[column].unique()
    print(f'Unique values in {column}: {unique_values}')

In [ ]:
df1['age'] = df['age'].replace(9, 1)
df1.to_csv('corrected_data.csv', index=False)



In [ ]:
df1.isna ().mean ()*100 # доля пропусков

Делаем вывод о большой доли пропусков в вышеуказанных столбцах.
Возникает вопрос, что делать с данными пропусками, так как заполнение пропусков в данном случае может исказить результаты (медицинские измерения). Но тем не менее посмотрим, как показатели зависят друг от друга. Например, как температура зависит от возраста, или пульс и частота дыхания от возраста лошади.

In [ ]:
df1.groupby ('age') ['rectal_temperature'].median ().sort_values ()

In [ ]:
df1.groupby ('pulse') ['rectal_temperature'].median ().sort_values ()

Зависимости от пульса не наблюдается.

In [ ]:
df1.groupby ('age') ['respiratory_rate'].median ().sort_values ()

Здесь же мы видим, что с возрастом увеличивается показатель частоты дыхания

In [ ]:
df1.groupby ('age') ['temperature_of_extremities'].median ().sort_values ()

Здесь мы также не наблюдаем зависимости температуры от возраста

In [ ]:
df1.groupby ('age') ['pulse'].median ().sort_values ()

С возрастом увеличивается пульс. Эти полученные данные мы будем использовать дальше для заполнения пропусков в столбцах.

In [ ]:
# ВОПРОС! # найдем выбросы, используя метод межквартильного размаха
def detect_outliers_iqr(selected_columns):
    Q1 = selected_columns.quantile(0.25)
    Q3 = selected_columns.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (selected_columns < lower_bound) | (selected_columns > upper_bound)

# Применяем функцию ко всем числовым столбцам
outliers = df1.apply(detect_outliers_iqr)

# Выводим результат
outliers_summary = df1[outliers.any(axis=1)]
print("Записи с выбросами:")
print(outliers_summary)

In [ ]:
# посмотрим количество пропусков
missing_data = df1.isnull().sum()
print("\nКоличество пропусков по столбцам:\n", missing_data)


In [ ]:
# ВОПРОС! # так как мы убедились, что с повышением возраста повышается пульс, то заполним пустые значения по этому столбцу
df1 ['pulse'].fillna (df1.groupby ('age')['pulse'].transform ('median'))

In [ ]:
df1.head()

In [ ]:
missing_data = df1.isnull().sum()
print("\nКоличество пропусков по столбцам:\n", missing_data)

In [ ]:
# построим визуализацию зависимости пульса от возраста
sns.boxplot(data = df1, x = 'age', y = 'pulse')
plt.title ('Зависимость пульса от возраста')
plt.show


In [ ]:
df1 ['respiratory_rate'].fillna (df1.groupby ('age')['respiratory_rate'].transform ('median'))

In [ ]:
# построим визуализацию зависимости частоты дыхания от пульса
sns.boxplot (data = df1, x = 'age', y = 'respiratory_rate')
plt.title ('Зависимость частоты дыхания от возраста')
plt.show

In [ ]:
df1['rectal_temperature'].fillna(df1.median, inplace=True)


In [ ]:
### ОБЩЕЕ заполнение пропусков. ДО этого не понимаю как работает, если добавлять в соотв. с взаимосвязью столбцов

# Заполнение пропусков
for selected_columns in df1.columns:
    if df1[selected_columns].dtype == 'float64' or df1[selected_columns].dtype == 'int64':
        df1[selected_columns] = df1[selected_columns].fillna(df1[selected_columns].median())  # Числовые заполняем медианой
    else:
        df1[selected_columns] = df1[selected_columns].fillna(df1[selected_columns].mode()[0])  # Категориальные заполняем модой

print("\nДатафрейм без пропусков:\n", df1.head())